# Capstone Assignment 2 - Toronto

### Part 1 - Dataframe

In [1]:
import pandas as pd
import numpy as np


In [17]:
df_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


In [105]:
df_tor = df_wiki[0]
df_tor.head()
df_tor1 = df_tor[df_tor.Borough != 'Not assigned']
df_tor1.reset_index(drop=True, inplace=True)

x = df_tor1.Neighbourhood == 'Not assigned'
y = x.value_counts()
y


False    103
Name: Neighbourhood, dtype: int64

N.B. There are no values of Neighbourhead that are equal to 'Not assigned'.

In [106]:
df_tor1

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [103]:
df_tor1.shape

(103, 3)